<a href="https://colab.research.google.com/github/enverakbacak/EMVR/blob/main/videoGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd

/content


In [3]:
cd drive/MyDrive/ColabNotebooks/video_generators/


/content/drive/MyDrive/ColabNotebooks/video_generators


In [4]:
import tensorflow as tf
import cv2
import numpy as np
import sys,os
import time
import matplotlib
import scipy.io
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow.keras.optimizers
from tensorflow.keras import Input 
from tensorflow.keras.models import Model
#Inputlayer
from tensorflow.keras.layers import Dense, Dropout, Flatten, TimeDistributed, Conv2D, MaxPool2D, LSTM, Bidirectional, BatchNormalization
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
import pandas as pd
import matplotlib.pyplot as plt
matplotlib.use('Agg')
from skimage.transform import resize   # for resizing images
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
import math

In [5]:
batch_size = 1
resize     = 224

def generator(samples, labels, batch_size=batch_size):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[image1_filename,label1], [image2_filename,label2],...].
    """
    num_samples = len(samples)
  
    while True: # Loop forever so the generator never terminates

        # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size &lt;= num_samples]
        for offset in range(0, num_samples, batch_size):
            # Get the samples you'll use in this batch
            batch_samples = samples[offset:offset+batch_size]
            label_samples = labels[offset:offset+batch_size]
            # Initialise X_train and y_train arrays for this batch
           
            y = []
            X = []
            # For each example
            for batch_sample in batch_samples:
              #filename = batch_sample
                cap = cv2.VideoCapture(batch_sample)
                fps = cap.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
                frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                duration = frame_count/fps
                N = 10 # How many frames will be extracted
                interwall = duration/N   #//it will capture image in each 'interwall' second
                           
                sec = 0                          
                frames = []
                for frame in range(0,N,1):
                  cap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
                  sec = sec + interwall
                  ret, frame = cap.read()
                  frame = cv2.resize(frame,(resize, resize))
                  frames.append(frame) 
                X.append(frames)
                                                       
            X = np.array(X)
            X.shape
            #print(X.shape[:])
            X = X.astype(float)
            X = X / 255
            #X = X.reshape(X.shape[0], X.shape[1] , X.shape[2] * X.shape[3] * X.shape[4])
            #print(X.shape[:])
                                   
            for label_sample in label_samples:
                y.append(label_sample)
            y = np.array(y)
            y = y.astype(float)
            #print(y.dtype)

            # The generator-y part: yield the next training batch
            yield X, y

In [6]:
data=[]
from csv import reader
# skip first line i.e. read header first and then iterate over each row od csv as a list
with open('trainlist01.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    header = next(csv_reader)
    # Check file as empty
    if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader:
            # row variable is a list that represents a row in csv
            data.append(row)
data=np.array(data)
videos = data[:,0]

labels = data[:,1]
labels = labels.astype(int)
labels_onehot = LabelBinarizer().fit_transform(labels)

In [7]:
train_generator = generator(videos, labels_onehot, batch_size=batch_size)

In [8]:
x,y = next(train_generator)
print ('data shape: ', x.shape[:])
print ('label shape: ', y.shape)
print(x.dtype)

data shape:  (1, 10, 224, 224, 3)
label shape:  (1, 101)
float64


In [9]:
hash_bits = 64

video_input = tensorflow.keras.Input(shape = (10,resize,resize,3), name='video')

base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg') # input_shape=(X.shape[1:])
base_model.trainable = False

frame_features = layers.TimeDistributed(base_model)(video_input)
blstm_1   = Bidirectional(LSTM(512, dropout=0.1, recurrent_dropout=0.5, return_sequences = True  ))(frame_features)
blstm_2   = Bidirectional(LSTM(512, dropout=0.1, recurrent_dropout=0.5, return_sequences = False ))(blstm_1)
batchNorm = BatchNormalization()(blstm_2)
Dense_1   = Dense(256)(batchNorm)
Dense_2   = Dense(hash_bits, activation = 'sigmoid' )(Dense_1)
#batchNorm = BatchNormalization()(Dense_2)
Dense_3   = Dense(101, activation='sigmoid')(Dense_2)
model     = Model(video_input, Dense_3)
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
video (InputLayer)           [(None, 10, 224, 224, 3)] 0         
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 2048)          21802784  
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 1024)          10489856  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1024)              6295552   
_________________________________________________________________
batch_normalization_94 (Batc (None, 1024)              4096      
_________________________________________________________________
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448 

In [10]:
#sgd = SGD(learning_rate=0.01, decay = 1e-6, momentum=0.9, nesterov=True)

def c_loss_1(y_true, y_pred):
    return  tf.keras.losses.binary_crossentropy(y_true, y_pred)

def c_loss_2(noise_1, noise_2):
    noise_1 = tf.cast(Dense_2 > 0.5 , tf.float32 )
    noise_2 = Dense_2
    return  tf.keras.losses.binary_crossentropy(noise_1, noise_2)


initial_learning_rate = 0.0001
lr_schedule = tensorflow.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

model.compile(loss = [c_loss_1, c_loss_2],  
              optimizer=tensorflow.keras.optimizers.Adam(learning_rate=lr_schedule),
              metrics=['accuracy'],
              )

In [ ]:
cb = [
      tensorflow.keras.callbacks.EarlyStopping(monitor="accuracy", 
                                    patience=15,
                                    mode='min',
                                    verbose=1),
      tensorflow.keras.callbacks.ModelCheckpoint(save_best_only=True,
                                      filepath='./models/model.{epoch:02d}-{val_loss:.2f}.h5'),
      tensorflow.keras.callbacks.TensorBoard(log_dir='./logs'),
     ]

EPOCHS=1000
model.fit_generator(train_generator, 
                    steps_per_epoch  = math.ceil(len(videos) // batch_size), 
                    #validation_data=val_generator,
                    #validation_steps = math.ceil(len(val_data) // batch_size),
                    verbose=1,
                    callbacks=cb, 
                    epochs=EPOCHS
                    )

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
 320/9536 [>.............................] - ETA: 1:04:28 - loss: 0.4230 - accuracy: 0.2062